In [0]:
%pip install xgboost
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import xgboost as xgb
import warnings

warnings.filterwarnings('ignore')

#==============================================================================
# CARGAR Y PROCESAR EL PRIMER ARCHIVO (DATOS DE ENTRENAMIENTO)
#==============================================================================

# --- Cargar archivo Excel ---
# En Python, puedes especificar la ruta directamente. No hay un 'file.choose()' interactivo
# de la misma manera que en R.
ruta_excel_1 = "C:\\Users\\ramos\\Descargas\\tarea\\produccion-2024-2.csv"
datos10 = pd.read_excel(ruta_excel_1, sheet_name='train2024-2')


In [0]:
# --- Exploración inicial de datos ---
print("Dimensiones de datos10:", datos10.shape)
print("\nColumnas de datos10:", datos10.columns)


In [0]:
# Verificar datos faltantes en 'fecha_nacimiento'
print("\nValores nulos en 'fecha_nacimiento':", datos10['fecha_nacimiento'].isnull().sum())


In [0]:
datos1 = datos10.copy()
print("\nTipos de datos iniciales:\n", datos1.dtypes)

In [0]:
# --- Procesamiento de fechas y cálculo de edad ---
datos1['fecha_nacimiento'] = pd.to_datetime(datos1['fecha_nacimiento'], format="%d/%m/%Y")
def calcular_edad(fecha_nacimiento, fecha_referencia):
    edad = fecha_referencia.year - fecha_nacimiento.year - ((fecha_referencia.month, fecha_referencia.day) < (fecha_nacimiento.month, fecha_nacimiento.day))
    return edad

In [0]:
fecha_de_corte_1 = pd.to_datetime("2023-08-01")
datos1['edad'] = datos1['fecha_nacimiento'].apply(lambda x: calcular_edad(x, fecha_de_corte_1))


In [0]:
# --- Conversión y recodificación de variables categóricas ---
datos1['TIPO_ASIGNATURA'] = datos1['TIPO_ASIGNATURA'].astype('category')
datos1.rename(columns={'STATUS': 'condicion_final'}, inplace=True)
datos1['condicion_final'] = datos1['condicion_final'].astype('category')

In [0]:
# Recodificar 'PROGRAMA'
datos1['PROGRAMA'] = datos1['PROGRAMA'].astype('category')
datos1['PROGRAMA'] = datos1['PROGRAMA'].cat.rename_categories({
    "ADMINISTRACIÓN Y NEGOCIOS DIGITALES": "ADMINISTRACIÓN Y MARKETING",
    "CIENCIA DE LA COMPUTACIÓN": "INGENIERÍA DE SISTEMAS E INFORMÁTICA",
    "EDUCACIÓN CON ESPECIALIDAD EN INNOVACIÓN Y APRENDIZAJE DIGITAL": "CIENCIAS Y TECNOLOGÍAS DE LA COMUNICACIÓN",
    "INGENIERÍA ELECTRÓNICA": "INGENIERÍA ELÉCTRICA"
})


In [0]:
# Recodificar 'CAMPUS'
datos1['CAMPUS'] = pd.Categorical(datos1['CAMPUS'],
                                  categories=["FILIAL AREQUIPA", "FILIAL CUSCO", "FILIAL LIMA", "SEDE HUANCAYO", "VIRTUAL"],
                                  ordered=False).rename_categories(["AREQUIPA", "CUSCO", "LIMA", "HUANCAYO", "VIRTUAL"])

# Recodificar 'MODALIDAD'

In [0]:
# Recodificar 'MODALIDAD'
datos1['MODALIDAD'] = pd.Categorical(datos1['MODALIDAD'],
                                     categories=["UC-A DISTANCIA", "UC-PRESENCIAL", "UC-SEMIPRESENCIAL"],
                                     ordered=False).rename_categories(["DISTANCIA", "PRESENCIAL", "SEMIPRESENCIAL"])

In [0]:
 --- Procesamiento de 'PERIODO_ADMISION' y 'CATALOGO' ---
datos1['PERIODO_ADMISION'] = pd.to_numeric(datos1['PERIODO_ADMISION'].astype(str).str.slice(0, 5))
datos1['PERIODO_ADMISION'] = np.where(datos1['PERIODO_ADMISION'] % 10 <= 1,
                                      datos1['PERIODO_ADMISION'] // 10,
                                      datos1['PERIODO_ADMISION'] // 10 + 0.5)
datos1['PERIODO_ADMISION'] = 2023.5 - datos1['PERIODO_ADMISION']

datos1['CATALOGO'] = pd.to_numeric(datos1['CATALOGO'].astype(str).str.slice(0, 4))
datos1['CATALOGO'] = np.select(
    [datos1['CATALOGO'] <= 2016, datos1['CATALOGO'] <= 2020],
    ['Malla antigua', 'Malla intermedia'],
    default='Malla actual'
)


In [0]:
# --- Selección de variables y conversión de tipos ---
columnas_a_eliminar_1 = [datos1.columns[i] for i in [0,1,8,9,11,12,13,14,16,28]]
datos1 = datos1.drop(columns=columnas_a_eliminar_1)

for col in datos1.select_dtypes(include=['object']).columns:
    datos1[col] = datos1[col].astype('category')

print("\nColumnas finales de datos1:", datos1.columns)
print("\nTipos de datos finales de datos1:\n", datos1.dtypes)

In [0]:
#==============================================================================
# CARGAR Y PROCESAR EL SEGUNDO ARCHIVO (DATOS DE PRODUCCIÓN)
#==============================================================================

ruta_excel_2 = "C:\\Users\\Sandro28\\Desktop\\U.Continental2023\\ASIGNATURAS CRITICAS\\PROYECTO ENTREGABLE\\2023-2 para 2024-2\\produccion 2024-2.xlsx"
datos20 = pd.read_excel(ruta_excel_2, sheet_name='produccion24-2')

datos2 = datos20.copy()

# --- Procesamiento de fechas y cálculo de edad ---
datos2['fecha_nacimiento'] = pd.to_datetime(datos2['fecha_nacimiento'], format="%d/%m/%Y")
fecha_de_corte_2 = pd.to_datetime("2024-08-01")
datos2['edad'] = datos2['fecha_nacimiento'].apply(lambda x: calcular_edad(x, fecha_de_corte_2))

# --- Recodificación de variables ---
datos2['TIPO_ASIGNATURA'] = datos2['TIPO_ASIGNATURA'].astype('category')
datos2['TIPO_ASIGNATURA'] = datos2['TIPO_ASIGNATURA'].cat.rename_categories({
    "General Ciencias": "General - Ciencias",
    "Transversal Ciencias": "General - Ciencias",
    "General Humanidades": "General - Humanidades",
    "Transversal Ciencias de Salud": "Facultad - Ciencias Salud",
    "Transversal Ciencias Empresa": "Facultad - Ciencias Empresa",
    "Transversal Humanidades": "Facultad - Humanidades",
    "Transversal Ingeniería": "Facultad - Ingeniería"
})

datos2['PROGRAMA'] = datos2['PROGRAMA'].astype('category')
datos2['PROGRAMA'] = datos2['PROGRAMA'].cat.rename_categories({
    "ADMINISTRACIÓN Y NEGOCIOS DIGITALES": "ADMINISTRACIÓN Y MARKETING",
    "CIENCIA DE LA COMPUTACIÓN": "INGENIERÍA DE SISTEMAS E INFORMÁTICA",
    "EDUCACIÓN CON ESPECIALIDAD EN INNOVACIÓN Y APRENDIZAJE DIGITAL": "CIENCIAS Y TECNOLOGÍAS DE LA COMUNICACIÓN"
})

datos2['CAMPUS'] = pd.Categorical(datos2['CAMPUS'],
                                  categories=["FILIAL AREQUIPA", "FILIAL CUSCO", "FILIAL LIMA", "SEDE HUANCAYO", "VIRTUAL"],
                                  ordered=False).rename_categories(["AREQUIPA", "CUSCO", "LIMA", "HUANCAYO", "VIRTUAL"])

datos2['MODALIDAD'] = pd.Categorical(datos2['MODALIDAD'],
                                     categories=["UC-A DISTANCIA", "UC-PRESENCIAL", "UC-SEMIPRESENCIAL"],
                                     ordered=False).rename_categories(["DISTANCIA", "PRESENCIAL", "SEMIPRESENCIAL"])

# --- Procesamiento de 'PERIODO_ADMISION' y 'CATALOGO' ---
datos2['PERIODO_ADMISION'] = pd.to_numeric(datos2['PERIODO_ADMISION'].astype(str).str.slice(0, 5))
datos2['PERIODO_ADMISION'] = np.where(datos2['PERIODO_ADMISION'] % 10 <= 1,
                                      datos2['PERIODO_ADMISION'] // 10,
                                      datos2['PERIODO_ADMISION'] // 10 + 0.5)
datos2['PERIODO_ADMISION'] = 2024.5 - datos2['PERIODO_ADMISION']

datos2['CATALOGO'] = pd.to_numeric(datos2['CATALOGO'].astype(str).str.slice(0, 4))
datos2['CATALOGO'] = np.select(
    [datos2['CATALOGO'] <= 2016, datos2['CATALOGO'] <= 2020],
    ['Malla antigua', 'Malla intermedia'],
    default='Malla actual'
)

In [0]:
# --- Selección de variables y conversión de tipos ---
columnas_a_eliminar_2 = [datos2.columns[i] for i in [0,1,8,9,11,12,13,14,16,26]]
datos2 = datos2.drop(columns=columnas_a_eliminar_2)

for col in datos2.select_dtypes(include=['object']).columns:
    datos2[col] = datos2[col].astype('category')

print("\nColumnas finales de datos2:", datos2.columns)


In [0]:
#==============================================================================
# PREPARACIÓN DE DATOS PARA EL MODELO
#==============================================================================

# --- División en entrenamiento y prueba ---
X = datos1.drop('condicion_final', axis=1)
y = datos1['condicion_final']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=50, stratify=y)

# Juntar para el submuestreo
train1 = pd.concat([X_train, y_train], axis=1)
test1 = pd.concat([X_test, y_test], axis=1)

# --- Balanceo de datos (Undersampling) ---
condicion_yes = train1[train1['condicion_final'] == 0]
condicion_no = train1[train1['condicion_final'] == 1]

# Submuestreo de la clase mayoritaria
condicion_no_downsampled = condicion_no.sample(n=int(len(condicion_yes) * 1.65), random_state=50)

# Combinar y barajar
condicion_down = pd.concat([condicion_no_downsampled, condicion_yes]).sample(frac=1, random_state=50).reset_index(drop=True)

print("\nDistribución de clases después del submuestreo:")
print(condicion_down['condicion_final'].value_counts())

In [0]:
# --- Preparación final para los modelos ---
# One-Hot Encoding para variables categóricas
X_down = pd.get_dummies(condicion_down.drop('condicion_final', axis=1), drop_first=True)
y_down = condicion_down['condicion_final']

X_test_processed = pd.get_dummies(test1.drop('condicion_final', axis=1), drop_first=True)
y_test_processed = test1['condicion_final']

# Alinear columnas entre entrenamiento y prueba
train_cols = X_down.columns
test_cols = X_test_processed.columns
missing_in_test = set(train_cols) - set(test_cols)
for c in missing_in_test:
    X_test_processed[c] = 0
missing_in_train = set(test_cols) - set(train_cols)
for c in missing_in_train:
    X_down[c] = 0
X_test_processed = X_test_processed[train_cols]

In [0]:
#==============================================================================
# MODELO XGBOOST
#==============================================================================
print("\n--- Entrenando modelo XGBoost ---")

# Convertir a DMatrix
dtrain = xgb.DMatrix(X_down, label=y_down)
dtest = xgb.DMatrix(X_test_processed, label=y_test_processed)

# Parámetros del modelo
params = {'objective': 'binary:logistic'}

# Entrenamiento
xgb_model = xgb.train(params=params,
                      dtrain=dtrain,
                      num_boost_round=200,
                      verbose_eval=False)

# Predicciones
predicciones = xgb_model.predict(dtest)
predicciones_binarias = (predicciones > 0.5).astype(int)

# Matriz de confusión
print("\nMatriz de Confusión para XGBoost:")
print(confusion_matrix(y_test_processed, predicciones_binarias))


#==============================================================================
# MODELO RANDOM FOREST
#==============================================================================
print("\n--- Entrenando modelo Random Forest ---")

# En scikit-learn, el manejo de NAs se debe hacer antes.
# 'na.roughfix' se puede emular imputando con la mediana/media.
# Como el preprocesamiento no mencionó NAs, asumimos que no hay.
rf_model = RandomForestClassifier(n_estimators=100,
                                  max_features=7,
                                  random_state=50,
                                  oob_score=True,
                                  class_weight='balanced')

rf_model.fit(X_down, y_down)

print("\nImportancia de las variables (Random Forest):")
importances = rf_model.feature_importances_
feature_importance_df = pd.DataFrame({'feature': X_down.columns, 'importance': importances})
print(feature_importance_df.sort_values(by='importance', ascending=False).head(10))

# --- Predicción en el conjunto de prueba (test1) ---
des_nbal_pred_RF_prob = rf_model.predict_proba(X_test_processed)[:, 1]
des_nbal_pred_clase_RF = (des_nbal_pred_RF_prob >= 0.55).astype(int)

print("\nDistribución de las predicciones (Random Forest):")
print(pd.Series(des_nbal_pred_clase_RF).value_counts())

print("\nMatriz de Confusión para Random Forest (umbral 0.55):")
print(confusion_matrix(y_test_processed, des_nbal_pred_clase_RF))


In [0]:
# --- Búsqueda del mejor umbral ---
def buscar_mejor_umbral(y_real, y_prob):
    umbrales = np.arange(0.05, 1.0, 0.05)
    metricas = []
    for u in umbrales:
        y_pred = (y_prob >= u).astype(int)
        f1 = f1_score(y_real, y_pred, pos_label=0) # Asumiendo '0' como clase positiva
        metricas.append({'umbral': u, 'f1_score': f1})
    
    metricas_df = pd.DataFrame(metricas)
    mejor_umbral = metricas_df.loc[metricas_df['f1_score'].idxmax()]
    return mejor_umbral, metricas_df

mejor_umbral_info, umbrales_df = buscar_mejor_umbral(y_test_processed, des_nbal_pred_RF_prob)

print("\nBúsqueda de umbrales para Random Forest:")
print(umbrales_df)
print("\nMejor umbral encontrado:")
print(mejor_umbral_info)


In [0]:
#==============================================================================
# APLICAR MODELO A DATOS DE PRODUCCIÓN Y EXPORTAR
#==============================================================================
print("\n--- Aplicando modelo a nuevos datos y exportando ---")

# Alinear columnas de datos2 para la predicción
datos2_processed = pd.get_dummies(datos2, drop_first=True)
train_cols = X_down.columns
prod_cols = datos2_processed.columns

missing_in_prod = set(train_cols) - set(prod_cols)
for c in missing_in_prod:
    datos2_processed[c] = 0

missing_in_train = set(prod_cols) - set(train_cols)
# Es importante eliminar columnas en producción que no estaban en entrenamiento
datos2_processed = datos2_processed.drop(columns=list(missing_in_train))
    
datos2_processed = datos2_processed[train_cols]

# Predicción en los nuevos datos (datos2)
probabilidades_prod = rf_model.predict_proba(datos2_processed)[:, 1]

# Usar el umbral fijo de 0.55 como en el código R
recomendacion_prod = (probabilidades_prod >= 0.55).astype(int)

# Añadir columnas al DataFrame original
datos20['probabilidades'] = probabilidades_prod
datos20['Recomendacion'] = recomendacion_prod

In [0]:
# --- Exportar resultados a Excel ---
try:
    datos20.to_excel("BIGDATA-FINAL-202420.xlsx", index=False)
    print("\nResultados de producción exportados a 'BIGDATA-FINAL-202420.xlsx'")
except Exception as e:
    print(f"\nError al exportar a Excel: {e}")


#==============================================================================
# CREAR Y EXPORTAR TABLA RESUMEN
#==============================================================================

tabla_resumen = datos10.groupby('NOMBRE_ASIGNATURA').apply(
    lambda x: pd.Series({
        'aprobados': x['STATUS'].sum(),
        'Total': len(x),
        'porc_apro': (x['STATUS'].sum() / len(x)) * 100
    })
).reset_index()

print("\nTabla Resumen 2024:")
print(tabla_resumen)

try:
    tabla_resumen.to_excel("TABLA2024-1.xlsx", index=False)
    print("\nTabla resumen exportada a 'TABLA2024-1.xlsx'")
except Exception as e:
    print(f"\nError al exportar la tabla resumen: {e}")
